# 🧠 AccessGen: AI Meal Planner with Gemini 2.0 + Nutrition Data 🍽️

This notebook demonstrates an end-to-end Generative AI system that helps users plan healthy meals through a combination of:

- 🤖 Conversational chatbot to estimate your daily nutritional needs.
- 🖼️ Image-based calorie and macronutrient recognition using **Gemini 2.0 Flash**.
- 📊 Nutrition info lookup from the **Food-101 Nutrition Dataset**.
- 📝 Estimated calories, macros (protein, carbs, fat), and running time to burn the meal.
- 🔄 Multilingual support and dynamic user interface with widgets.

---

This project was built for the **Google Gen AI Intensive Capstone (Q1 2025)**.


In [9]:
# 📦 Required Libraries

# 🔧 Core
import os
import re
import json
import random

# 📊 Data & Visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 🖼️ Image Processing
from PIL import Image
from base64 import b64encode

# 💡 Interactive UI
import ipywidgets as widgets

# 📡 API & Requests
import requests

# 📎 Kaggle Secrets (API key access)
from kaggle_secrets import UserSecretsClient

# 🧠 Jupyter Display Utilities
from IPython.display import display, Markdown, clear_output
from IPython.display import Image as IPyImage


# 📦 Install & Authenticate Gemini 2.0 API

This cell installs required packages and authenticates the Gemini 2.0 API using your secret key stored in `GOOGLE_API_KEY` Kaggle secret.

You must create a Gemini API key via [https://makersuite.google.com/app/apikey](https://makersuite.google.com/app/apikey) and add it in Kaggle → Secrets.


In [10]:
# 📌 Configuration Constants
from kaggle_secrets import UserSecretsClient

# 🔐 Gemini API setup
KAGGLE_SECRET_KEY_NAME = "GOOGLE_API_KEY"
GEMINI_API_KEY = UserSecretsClient().get_secret(KAGGLE_SECRET_KEY_NAME)
GEMINI_MODEL_NAME = "models/gemini-2.0"
GEMINI_API_URL = f"https://generativelanguage.googleapis.com/v1beta/{GEMINI_MODEL_NAME}:generateContent?key={GEMINI_API_KEY}"

# 🌐 Default request headers
DEFAULT_HEADERS = {"Content-Type": "application/json"}



# 🌍 Language Selection

Use the dropdown below to choose your preferred output language. All text and audio output will follow this setting.

This function is not fully implemented in the model so just choose English


In [11]:
# 🌍 Multilingual Support

languages = {
    "English": "en", "Arabic": "ar", "Turkish": "tr",
    "Spanish": "es", "French": "fr", "German": "de",
    "Italian": "it", "Hindi": "hi"
}

lang_dropdown = widgets.Dropdown(
    options=list(languages.keys()), value="English", description="🌐 Language:"
)

display(Markdown("### 🌐 Select Your Language"))
display(lang_dropdown)

chosen_lang = lang_dropdown.value
lang_code = languages[chosen_lang]


### 🌐 Select Your Language

Dropdown(description='🌐 Language:', options=('English', 'Arabic', 'Turkish', 'Spanish', 'French', 'German', 'I…

In [ ]:
import ipywidgets as widgets
from IPython.display import display

languages = {
    "English": "en",
    "Arabic": "ar",
    "Turkish": "tr",
    "Spanish": "es",
    "French": "fr",
    "German": "de",
    "Hindi": "hi",
    "Chinese": "zh"
}

lang_dropdown = widgets.Dropdown(
    options=list(languages.keys()),
    value="English",
    description="🌐 Language:"
)

display(lang_dropdown)

chosen_lang = lang_dropdown.value
lang_code = languages[chosen_lang]


# Dataset


In [12]:
# 📥 Load Food Nutrition Dataset (Food-101 style)
nutrition_df = pd.read_csv("/kaggle/input/food-101-nutritional-information/nutrition.csv")

# 📊 Show sample
display(Markdown("### 📊 Sample Nutrition Data"))
display(nutrition_df.head())


### 📊 Sample Nutrition Data

,label,weight,calories,protein,carbohydrates,fats,fiber,sugars,sodium
0,apple_pie,80,240,2,36,10,2,16,120
1,apple_pie,100,300,3,45,12,2,20,150
2,apple_pie,120,360,4,54,14,3,24,180
3,apple_pie,150,450,5,68,18,3,30,225
4,apple_pie,200,600,6,90,24,4,40,300


# 🍽️ Smart Nutrient-Based Food Recommender From Target Nutrients
This Python tool helps users find foods that match their nutritional goals by:

📥 Loading a dataset of foods with detailed nutrient info

🧾 Asking users to input desired values for nutrients (e.g., calories, protein, etc.)

🧮 Calculating the similarity between each food and the user’s targets using Euclidean distance

🏆 Recommending the top 5 closest-matching foods

In [13]:
# Step 1: Ask the user what nutrients they care about
nutrient_columns = ["calories", "protein", "carbohydrates", "fats", "fiber", "sugars", "sodium"]
desired_values = {}

print("🧠 Let's find foods that match your target nutrients.")
for nutrient in nutrient_columns:
    value = input(f"Enter desired amount of {nutrient} (in grams or mg for sodium): ").strip()
    try:
        desired_values[nutrient] = float(value)
    except ValueError:
        desired_values[nutrient] = float(0)
        print(f"⚠️ No input for {nutrient}, taking 0...")

# Step 2: Calculate distance to each food
def calculate_similarity(df, targets):
    df = df.copy()
    for nutrient in targets:
        if nutrient not in df.columns:
            raise ValueError(f"Nutrient {nutrient} not found in dataset.")
    
    # Compute Euclidean distance between target and each food item
    def distance(row):
        return np.sqrt(sum((row[n] - targets[n]) ** 2 for n in targets))
    
    df["distance"] = df.apply(distance, axis=1)
    return df.sort_values("distance")

# Step 3: Run matching algorithm and display top results
if desired_values:
    matched_df = calculate_similarity(nutrition_df, desired_values)
    top_matches = matched_df.head(5)
    display(Markdown("### 🍽️ Top Foods Closest to Your Nutrient Targets"))
    display(top_matches[["label"] + list(desired_values.keys()) + ["distance"]])
else:
    print("❌ No nutrient preferences provided.")


🧠 Let's find foods that match your target nutrients.


Enter desired amount of calories (in grams or mg for sodium):  


⚠️ No input for calories, taking 0...


Enter desired amount of protein (in grams or mg for sodium):  


⚠️ No input for protein, taking 0...


...

Enter desired amount of fats (in grams or mg for sodium):  768
Enter desired amount of fiber (in grams or mg for sodium):  


⚠️ No input for fiber, taking 0...


Enter desired amount of sugars (in grams or mg for sodium):  545
Enter desired amount of sodium (in grams or mg for sodium):  67


### 🍽️ Top Foods Closest to Your Nutrient Targets

,label,calories,protein,carbohydrates,fats,fiber,sugars,sodium,distance
255,guacamole,100,2,5,8,3,1,50,940.139883
225,frozen_yogurt,150,4,25,3,0,20,50,940.361633
150,deviled_eggs,100,6,1,8,0,1,120,941.478624
285,hummus,100,4,8,6,3,1,100,942.209106
10,baklava,165,3,20,8,1,13,60,942.500928


In [14]:
desired_values

{'calories': 0.0,
 'protein': 0.0,
 'carbohydrates': 0.0,
 'fats': 768.0,
 'fiber': 0.0,
 'sugars': 545.0,
 'sodium': 67.0}

# 🖼️ Step 2: Select a Food Image

Choose a sample food image from the uploaded dataset to analyze its contents and estimate the calories.


In [15]:

# ✅ Set path to food41 image folder
image_root = "/kaggle/input/food41/images"

# ✅ Get one random image from each category (limit to 9 for UI)
category_dirs = sorted([os.path.join(image_root, d) for d in os.listdir(image_root) if os.path.isdir(os.path.join(image_root, d))])
sample_paths = [os.path.join(cat, random.choice(os.listdir(cat))) for cat in category_dirs[:9]]

# ✅ Global selection variable
selected_image_path = None

# ✅ Handle click event
def on_image_click(path):
    def handler(btn):
        global selected_image_path
        selected_image_path = path
        clear_output(wait=True)
        display(Markdown(f"### ✅ You selected: `{os.path.basename(path)}`"))
        display(Image.open(path))
    return handler

# ✅ Convert images to base64 for display in ipywidgets.Image
def image_to_bytes(path):
    with open(path, "rb") as f:
        return f.read()

# ✅ Build grid with images + buttons
widget_rows = []
for path in sample_paths:
    image_bytes = image_to_bytes(path)
    img_widget = widgets.Image(
        value=image_bytes,
        format='jpg',
        width=150,
        height=150
    )
    button = widgets.Button(description=os.path.basename(path)[:20], layout=widgets.Layout(width="160px"))
    button.on_click(on_image_click(path))
    widget_rows.append(widgets.VBox([img_widget, button]))

# ✅ Display image selection grid
display(Markdown("### 🍽️ Choose a food image to analyze:"))
grid = widgets.GridBox(widget_rows, layout=widgets.Layout(grid_template_columns="repeat(3, 200px)"))
display(grid)


### 🍽️ Choose a food image to analyze:

GridBox(children=(VBox(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\…

In [16]:
selected_image_path

'/kaggle/input/food41/images/baby_back_ribs/3010421.jpg'

# 🔍 Step 3: Gemini Vision - Estimate Calories

The selected image will be analyzed using Gemini 2.0 Vision to estimate total calorie content and describe the food shown.


In [17]:

# 🔐 Load API key
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# 🔁 Convert image to base64
def image_to_base64(path):
    with open(path, "rb") as f:
        return b64encode(f.read()).decode("utf-8")

# 🧠 Analyze food image with Gemini
def analyze_selected_image(path, language="English"):
    if not path:
        return display(Markdown("⚠️ No image selected."))

    image_b64 = image_to_base64(path)
    
    # 📝 Prompt
    prompt = (
        f"Estimate the calories in this food image. "
        f"Reply in JSON format like: {{'food': '...', 'estimated_calories': ..., "
        f"'macros': {{'protein': ..., 'carbs': ..., 'fat': ...}}, "
        f"'micros': {{'fiber': ..., 'sugar': ..., 'sodium': ...}}, "
        f"'portion_grams': 100}}. Respond in {language}."
    )

    # 🔗 Gemini request setup
    headers = {"Content-Type": "application/json"}
    api_url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={GOOGLE_API_KEY}"

    payload = {
        "contents": [
            {
                "role": "user",
                "parts": [
                    {"text": prompt},
                    {
                        "inline_data": {
                            "mime_type": "image/jpeg",
                            "data": image_b64
                        }
                    }
                ]
            }
        ]
    }

    response = requests.post(api_url, headers=headers, json=payload)
    
    if response.status_code != 200:
        return display(Markdown(f"❌ API Error: {response.status_code}\n{response.text}"))

    text_result = response.json()["candidates"][0]["content"]["parts"][0]["text"]

    # 🧠 Parse JSON from text
    match = re.search(r"\{.*\}", text_result, re.DOTALL)
    if not match:
        return display(Markdown("⚠️ Could not parse JSON from Gemini.\n\n```json\n" + text_result + "\n```"))

    try:
        result = json.loads(match.group())

        food = result.get("food", "Unknown")
        calories = float(result.get("estimated_calories", 0))
        macros = result.get("macros", {})
        micros = result.get("micros", {})
        portion = result.get("portion_grams", 100)

        # 🏃 Burn time estimate
        burn_minutes = int(calories / 10)

        # 📊 Output summaries
        display(Markdown(f"### 🍽️ Food: **{food}**"))
        display(Markdown(f"🔥 **Calories:** `{int(calories)} kcal`"))
        display(Markdown(f"🏃 **Time to burn (running):** ~`{burn_minutes} min`"))
        display(Markdown(f"📏 **per** : `{portion} g`"))

        # 📋 Nutrients Table with units
        nutrient_data = {
            "Nutrient": [
                "Calories",
                "Protein (g)",
                "Carbohydrates (g)",
                "Fat (g)",
                "Fiber (g)",
                "Sugars (g)",
                "Sodium (mg)"
            ],
            "Value": [
                f"{int(calories)} kcal",
                f"{int(macros.get('protein', 0))} g",
                f"{int(macros.get('carbs', 0))} g",
                f"{int(macros.get('fat', 0))} g",
                f"{int(micros.get('fiber', 0))} g",
                f"{int(micros.get('sugar', 0))} g",
                f"{int(micros.get('sodium', 0))} mg"
            ]
        }

        nutrient_df = pd.DataFrame(nutrient_data)

        styled_table = nutrient_df.style.set_table_styles([
            {'selector': 'thead', 'props': [('background-color', '#222'), ('color', 'black'), ('font-weight', 'bold')]},
            {'selector': 'tbody', 'props': [('background-color', '#111'), ('color', 'black')]},
            {'selector': 'th', 'props': [('border', '1px solid #444'), ('padding', '8px')]},
            {'selector': 'td', 'props': [('border', '1px solid #444'), ('padding', '8px')]}
        ]).set_properties(**{'text-align': 'left'}).hide(axis="index")

        display(Markdown("### 🧪 Nutritional Information"))
        display(styled_table)

    except Exception as e:
        display(Markdown("❌ Parsing failed."))
        print(e)
        print("🔎 Raw output:\n```json\n" + text_result + "\n```")

# 🚀 Run this only after user selects an image and language
analyze_selected_image(selected_image_path, language=chosen_lang)


### 🍽️ Food: **Ribs (with sauce)**

🔥 **Calories:** `650 kcal`

🏃 **Time to burn (running):** ~`65 min`

📏 **per** : `250 g`

### 🧪 Nutritional Information

Nutrient,Value
Calories,650 kcal
Protein (g),45 g
Carbohydrates (g),20 g
Fat (g),45 g
Fiber (g),1 g
Sugars (g),15 g
Sodium (mg),700 mg


# 🍽️ Step 4: Generate Personalized Meal Plan

Based on your estimated calorie needs, Gemini 2.0 will generate a full meal plan with nutritional breakdown and dietary filters applied.


## 🗣️ AI Chat Assistant: Collecting Your Preferences

In this section, the assistant will ask you a few questions about your dietary needs and preferences (e.g., calories, goals, food restrictions). Your answers will be used to generate a personalized meal plan that meets your nutritional goals.


In [18]:
# 💬 Dynamic Chatbot for Collecting User Preferences
user_preferences = user_preferences if 'user_preferences' in locals() else {}

# 🔑 Define expected keys and prompts
preference_questions = {
    "goal": "What is your fitness goal? (lose, maintain, gain)",
    "diet": "Do you follow any specific diet? (e.g., vegetarian, keto, halal, none)",
    "allergies": "Do you have any allergies or food restrictions? (e.g., peanuts, gluten, lactose, none)",
    "favorite_foods": "Any favorite foods you'd like included?",
    "disliked_foods": "Any foods you dislike or want to avoid?",
    "timeline": "How long should the meal plan cover? (1 day, 1 week, 1 month, 3 months, 6 months)"
}

# 🧠 Ask only for missing keys
for key, question in preference_questions.items():
    if key not in user_preferences or not user_preferences[key]:
        response = input(question + " ").strip()
        user_preferences[key] = response.lower()

# ✅ Show collected preferences
print("\n📋 Your Collected Preferences:")
for k, v in user_preferences.items():
    print(f"{k.capitalize()}: {v}")


What is your fitness goal? (lose, maintain, gain)  
Do you follow any specific diet? (e.g., vegetarian, keto, halal, none)  
Do you have any allergies or food restrictions? (e.g., peanuts, gluten, lactose, none)  
Any favorite foods you'd like included?  
Any foods you dislike or want to avoid?  
How long should the meal plan cover? (1 day, 1 week, 1 month, 3 months, 6 months)  



📋 Your Collected Preferences:
Goal: 
Diet: 
Allergies: 
Favorite_foods: 
Disliked_foods: 
Timeline: 


In [ ]:
# 💬 Define chatbot questions to collect preferences
# 🧮 Estimate Daily Caloric Needs via BMR + Goal
age = int(input("Enter your age: "))
gender = input("Enter your gender (M/F): ").upper()
weight = float(input("Enter your weight in kg: "))
height = float(input("Enter your height in cm: "))
activity_level = input("Activity level? (sedentary, light, moderate, active, very active): ").lower()
goal = input("Goal? (lose, maintain, gain): ").lower()

# 🔢 BMR using Mifflin-St Jeor Equation
if gender == "M":
    bmr = 10 * weight + 6.25 * height - 5 * age + 5
else:
    bmr = 10 * weight + 6.25 * height - 5 * age - 161

# 🔁 Activity multiplier
activity_factors = {
    "sedentary": 1.2, "light": 1.375, "moderate": 1.55,
    "active": 1.725, "very active": 1.9
}
calories = bmr * activity_factors.get(activity_level, 1.2)

# 🎯 Adjust for goal
if goal == "lose":
    calories -= 500
elif goal == "gain":
    calories += 500

print(f"\n✅ Your estimated daily caloric need: {int(calories)} kcal")



Enter your age:  453
Enter your gender (M/F):  m
Enter your weight in kg:  65


# 📊 Step 5: Output Summary & Nutrition Display

Using your answers, Gemini will generate a custom meal plan that includes meals and snacks tailored to your dietary needs, goals, and preferences. The output includes:

- Meals and snacks with calorie breakdowns
- Macronutrient distribution
- Options to match your dietary needs

In [11]:

# 🔐 API Key
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
calorie_target = int(calories)

# ✨ Prompt for 14 days
meal_prompt = (
    f"You are a nutritionist AI assistant. The user needs **{calorie_target} kcal/day**.\n"
    f"Generate a JSON meal plan for 14 days ONLY.\n\n"
    f"User Preferences:\n"
    f"- Goal: {user_preferences.get('goal')}\n"
    f"- Diet: {user_preferences.get('diet')}\n"
    f"- Allergies: {user_preferences.get('allergies')}\n"
    f"- Favorite Foods: {user_preferences.get('favorite_foods')}\n"
    f"- Disliked Foods: {user_preferences.get('disliked_foods')}\n\n"
    f"Each day must include:\n"
    f"- Four meals (breakfast, lunch, dinner, snacks)\n"
    f"- Each meal includes:\n"
    f"   - dish_name\n"
    f"   - portion_size (e.g., 1 bowl, 300g)\n"
    f"   - nutrition: calories, protein, carbs, fat (as numbers or strings)\n"
    f"- A daily_nutrition summary (with numerical totals)\n\n"
    f"Reply in valid JSON under the top-level key: 'meal_plan'."
)

# 🌐 Gemini API call
headers = {"Content-Type": "application/json"}
api_url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={GOOGLE_API_KEY}"
data = {
    "contents": [ { "role": "user", "parts": [ {"text": meal_prompt} ] } ]
}

response = requests.post(api_url, headers=headers, json=data)
if response.status_code == 200:
    text_result = response.json()["candidates"][0]["content"]["parts"][0]["text"]
else:
    print("❌ Gemini API error:", response.status_code)
    print(response.text)
    text_result = ""

# 🔍 Extract and format
match = re.search(r"\{.*\}", text_result, re.DOTALL)
if not match:
    print("❌ No valid JSON found.")
    print(text_result)
else:
    try:
        meal_data = json.loads(match.group())
        days = meal_data["meal_plan"]
        extended_days = [(f"Day {i+1}", days[i % 14]) for i in range(28)]
        display(Markdown("## 📆 28-Day Personalized Meal Plan"))

        # 🔧 Helpers
        def extract_num(val):
            try:
                if isinstance(val, (int, float)): return round(val, 2)
                return round(float(re.findall(r"[\d.]+", str(val))[0]), 2)
            except: return 0

        def format_number(val):
            val = extract_num(val)
            return int(val) if val == int(val) else round(val, 1)

        def estimate_grams(portion):
            portion = portion.lower()
            if "bowl" in portion: return "~200g"
            if "plate" in portion: return "~350g"
            if "slice" in portion: return "~80g"
            if "cup" in portion: return "~240g"
            if "egg" in portion: return "~50g"
            if "serving" in portion: return "~150g"
            if re.search(r"\d+\s*g", portion): return portion
            return portion + " (~250g)"

        for label, content in extended_days:
            display(Markdown(f"### 🗓️ {label}"))
            meal_rows = []

            for meal in ["breakfast", "lunch", "dinner", "snacks"]:
                info = content.get(meal, {})

                # Handle snacks as list or dict
                if meal == "snacks" and isinstance(info, list):
                    combined_dish = []
                    total_nutrition = {"calories": 0, "protein": 0, "carbs": 0, "fat": 0}
                    total_portion = []

                    for snack in info:
                        combined_dish.append(snack.get("dish_name", "Snack"))
                        total_portion.append(snack.get("portion_size", ""))
                        n = snack.get("nutrition", {})
                        total_nutrition["calories"] += extract_num(n.get("calories", 0))
                        total_nutrition["protein"] += extract_num(n.get("protein", 0))
                        total_nutrition["carbs"] += extract_num(n.get("carbs", 0))
                        total_nutrition["fat"] += extract_num(n.get("fat", 0))

                    meal_rows.append({
                        "Meal": meal.capitalize(),
                        "Dish": ", ".join(combined_dish),
                        "Portion": ", ".join(total_portion),
                        "Calories (kcal)": format_number(total_nutrition["calories"]),
                        "Protein (g)": format_number(total_nutrition["protein"]),
                        "Carbs (g)": format_number(total_nutrition["carbs"]),
                        "Fat (g)": format_number(total_nutrition["fat"]),
                    })

                else:
                    nutrition = info.get("nutrition", {})
                    portion_raw = info.get("portion_size", "N/A")
                    meal_rows.append({
                        "Meal": meal.capitalize(),
                        "Dish": info.get("dish_name", "N/A"),
                        "Portion": estimate_grams(portion_raw),
                        "Calories (kcal)": format_number(nutrition.get("calories", 0)),
                        "Protein (g)": format_number(nutrition.get("protein", 0)),
                        "Carbs (g)": format_number(nutrition.get("carbs", 0)),
                        "Fat (g)": format_number(nutrition.get("fat", 0)),
                    })

            df = pd.DataFrame(meal_rows)
            display(df.style.set_caption("🍽️ Nutrition per Meal").hide(axis="index"))

            total = content.get("daily_nutrition", {})
            display(Markdown("**🧮 Daily Totals:**"))
            display(Markdown(
                f"- 🔥 Calories: `{format_number(total.get('calories', 0))} kcal`\n"
                f"- 💪 Protein: `{format_number(total.get('protein', 0))} g`\n"
                f"- 🍞 Carbs: `{format_number(total.get('carbs', 0))} g`\n"
                f"- 🧈 Fat: `{format_number(total.get('fat', 0))} g`"
            ))
            print("")

    except Exception as e:
        print("⚠️ Error parsing Gemini output.")
        print("Error:", e)
        print("Raw Output:\n", text_result)


## 📆 28-Day Personalized Meal Plan

### 🗓️ Day 1

Meal,Dish,Portion,Calories (kcal),Protein (g),Carbs (g),Fat (g)
Breakfast,N/A,n/a (~250g),0,0,0,0
Lunch,N/A,n/a (~250g),0,0,0,0
Dinner,N/A,n/a (~250g),0,0,0,0
Snacks,N/A,n/a (~250g),0,0,0,0


**🧮 Daily Totals:**

- 🔥 Calories: `1800 kcal`
- 💪 Protein: `78 g`
- 🍞 Carbs: `235 g`
- 🧈 Fat: `59 g`

...

- 🔥 Calories: `1800 kcal`
- 💪 Protein: `100 g`
- 🍞 Carbs: `210 g`
- 🧈 Fat: `55 g`

### 🗓️ Day 28

Meal,Dish,Portion,Calories (kcal),Protein (g),Carbs (g),Fat (g)
Breakfast,N/A,n/a (~250g),0,0,0,0
Lunch,N/A,n/a (~250g),0,0,0,0
Dinner,N/A,n/a (~250g),0,0,0,0
Snacks,N/A,n/a (~250g),0,0,0,0


**🧮 Daily Totals:**

- 🔥 Calories: `1800 kcal`
- 💪 Protein: `102 g`
- 🍞 Carbs: `190 g`
- 🧈 Fat: `65 g`

# 📥 Optional Export

You can export the final meal plan to a CSV file or PDF file or JSON for future use or sharing.


## ✅ Conclusion

This AI-powered meal planning system successfully integrates **image-based food recognition** and **personalized dietary planning** using Google’s **Gemini 2.0 Vision API**.

Through this notebook, the system:

- Accurately estimated nutritional values of food items using real food images.
- Calculated the user's **daily caloric needs** using the Mifflin-St Jeor BMR equation based on age, gender, weight, height, activity level, and fitness goals.
- Collected **dietary preferences** including allergens, diet type, and favorite/disliked foods through a conversational chatbot.
- Generated a 28-day personalized meal plan by repeating a 14-day cycle, fulfilling daily calorie targets. While the user could specify custom durations (1 week, 1 month, etc.), the current implementation is fixed to a 28-day output as a proof-of-concept. Future work could dynamically adjust the plan length based on user-selected timelines. which:
  - Met the user's target calories per day.
  - Included dish names, estimated portion sizes, and per-meal nutritional breakdowns.
  - Provided structured tables and summaries for each day.

---

## 💡 Future Work and Recommendations

To take this project further, we recommend the following enhancements:

- 🔄 **User Image Upload Support:**  
  Kaggle currently doesn't support direct image upload at runtime. Migrating to Google Colab or a custom web interface could allow users to upload and analyze their own food images.

- 🛒 **Shopping List Generator:**  
  Extract ingredients from meal plan dishes and compile a **weekly grocery list**, improving usability.

- 📱 **Mobile Integration:**  
  Package this solution into a **mobile app** with camera access for real-time calorie estimation and nutrition logging.

- 🗣️ **Multilingual Support + Voice Assistant:**  
  Add natural voice input/output to make the assistant more accessible, especially for visually impaired users.

- 📊 **Nutrient Tracking Dashboard:**  
  Add analytics showing total macros across the 28-day plan and compare them to WHO or FDA guidelines.

- 🤖 **AI Fine-Tuning or Embedding Use:**  
  Fine-tune a smaller open-source model using food-specific datasets (e.g., Food-101) for edge deployment on devices like Raspberry Pi.

  ## 🤖 Gen AI Capabilities Audit

| **Capability**                   | **Used?** | **Where? (Cell/Section)**                                  | **How It’s Used**                                                                                       |
|----------------------------------|-----------|-------------------------------------------------------------|----------------------------------------------------------------------------------------------------------|
| ✅ Structured Output / JSON       | Yes       | Gemini meal planning (Final prompt + table rendering)       | Gemini is prompted to return structured JSON with nested keys for daily nutrition and per-meal macros.  |
| ✅ Image Understanding            | Yes       | Image analysis with Gemini Vision (`analyze_selected_image`) | Uses Gemini Vision to analyze a selected food image and estimate calories/macros from visual data.      |
| ✅ Prompt Engineering             | Yes       | Meal planner prompt, calorie image prompt                   | Carefully crafted prompts instruct Gemini to reply in a specific format, include nutrients, and stay within caloric bounds. |
| 🟡 Grounding                      | Partial   | User calorie estimation used in prompt                      | BMR + activity levels are used as grounding for calorie budget, but not deeply integrated with external verification (e.g., real food dataset). |
| 🟡 Gen AI Evaluation              | No        | —                                                           | No automated evaluation metrics for Gemini output correctness or consistency.                            |
| 🟡 Document Understanding         | No        | —                                                           | No PDFs or external docs parsed.                                                                         |
| 🟡 Function Calling               | No        | —                                                           | Gemini doesn’t call any external function dynamically.                                                   |
| 🟡 Embeddings + Vector Search     | No        | —                                                           | Could be used for food similarity or grouping in nutrition DB.                                           |
| 🟡 RAG                            | No        | —                                                           | Gemini is not enhanced with retrieval-augmented generation.                                              |
| 🟡 Agents                         | No        | —                                                           | Could automate workflow from intake → prediction → meal generation.                                     |
| 🟡 MLOps / Gen AI Pipelines       | No        | —                                                           | No deployment, model tracking, or logging pipeline.                                                      |
| 🟡 Long Context / Caching         | No        | —                                                           | Not explicitly tested or handled.                                                                        |

---

## : Recommendations

### 🧠 Best Unused Gen AI Features To Add

| **Feature**     | **Why It's a Fit**                                                                 | **How to Implement**                                                                                   |
|------------------|-------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------|
| 🟢 RAG            | Improve response accuracy by referencing real nutrition data                      | Link food names returned from Gemini to Food-101 DB to correct hallucinated macros.                     |
| 🟢 Embeddings     | Cluster similar foods, provide substitutions for allergies/preferences            | Use embeddings from dish name descriptions, e.g., OpenAI embeddings + FAISS.                             |
| 🟢 Gen AI Eval    | Validate consistency in daily calorie sums or check hallucinations                 | Compare Gemini output with summed macros or known food labels.                                           |

---

> 🧠 This notebook combines real-time AI interaction, personalized health analytics, and structured dataset utilization to demonstrate a strong use case for generative AI in nutrition and well-being.
